In [15]:
# input data explorer
import os, sys
import numpy as np
import tensorflow as tf

# configure Tensorflow
session_conf = tf.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1)
session_conf.gpu_options.allow_growth=True

# intialize TF
tf.reset_default_graph()
sess = tf.Session(graph=tf.get_default_graph(), config=session_conf)
init = tf.global_variables_initializer()
sess.run(init)

# iterate through data 

In [16]:
# import stuff.
from fcvaegan import (
    Model,
    MODEL_DIR,
    INPUT_TENSOR_NAME,
    SIGNATURE_NAME,
    read_and_decode,
    PARAMS,
    W,H,C,CY,
    TRAIN_DIR,
    NUM_EXAMPLES_TRAIN,
    NUM_EXAMPLES_VALIDATION,
    NUM_EXAMPLES_TEST,
)
params = dict(PARAMS)
params.update({
    'is_training':True, # this impact inferences....due to batch norm...?
    'batch_size':4, # <<--- 4 is good for 6gb gpu.
})

batch_size = 1

def _input_fn(training_dir, training_filename, batch_size=batch_size):
    test_file = os.path.join(training_dir, training_filename)
    filename_queue = tf.train.string_input_producer([test_file])

    image, label = read_and_decode(filename_queue)
    images, labels = tf.train.batch(
        [image, label], batch_size=batch_size,
        capacity=1000 + 10 * batch_size)
    return {INPUT_TENSOR_NAME: images}, labels


#def eval_input_fn(training_dir=TRAIN_DIR, batch_size=batch_size, params=None):
#    return _input_fn(training_dir, 'validation.tfrecords', batch_size=batch_size)
#tf_images,tf_labels = eval_input_fn(batch_size=batch_size)

def train_input_fn(training_dir=TRAIN_DIR, batch_size=batch_size, params=None):
    return _input_fn(training_dir, 'train.tfrecords', batch_size=batch_size)
tf_images,tf_labels = train_input_fn(batch_size=batch_size)

In [17]:
coord = tf.train.Coordinator()
threads = tf.train.start_queue_runners(coord=coord,sess=sess)    


In [19]:
import matplotlib.pyplot as plt
%matplotlib inline

In [31]:
import yaml
with open('/media/external/scisoft/fc-vae-gan/data/label.yml','r') as f:
    label_dict = yaml.load(f.read())
max_labe_num = len(label_dict)

In [32]:
total_count,bins = np.histogram([],bins=max_labe_num,range=(0,max_labe_num))
total_count = np.array(total_count)
for n in range(NUM_EXAMPLES_TRAIN):
    img, lbl = sess.run([tf_images,tf_labels])
    count,bins = np.histogram(list((lbl.ravel())),bins=max_labe_num,range=(0,max_labe_num))
    total_count += np.array(count)

In [44]:
for ind in np.argwhere(total_count>np.percentile(total_count,90)):
    print(label_dict[ind[0]])

aeroplane
bed
bedclothes
bicycle
bird
boat
bottle
building
bus
cabinet
car
cat
ceiling
chair
cloth
cow
curtain
dog
door
fence
floor
grass
ground
horse
motorbike
mountain
person
picture
pottedplant
road
rock
rug
sheep
shelves
sky
snow
sofa
table
track
train
tree
tvmonitor
unknown
wall
water
window


In [45]:
STOP = False
if STOP:
    coord.request_stop()
    # Wait for threads to stop
    coord.join(threads)